## Estructura de la Aplicación:

- **Verificación del backup:**
    - Detectar la presencia del archivo DBSIF.FBK.
    - Verificar la fecha y hora de creación/modificación.
    - Obtener el tamaño del archivo.

- **Notificación por Email y/o WhatsApp:**
    - Si el backup es exitoso (el archivo cumple con los criterios), se envía una notificación.
    - En caso de fallo (si no se genera el archivo o tiene un tamaño inesperado), enviar una alerta.

**Requisitos:**

- Bibliotecas necesarias:
    - `os` para verificar el archivo.
    - `smtplib` para enviar emails.
    - API de WhatsApp (podemos usar Twilio o alguna otra plataforma que permita la integración).
    - `schedule` o utilizar una tarea programada en Windows para ejecutar la verificación.

## Verificación del archivo DBSIF.FBK:


In [23]:
import os
import time

def verificar_backup(ruta_backup):
    # Verificar si el archivo existe
    if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB
        tamano = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        # Obtener fecha de creación/modificación
        timestamp = os.path.getmtime(ruta_backup)
        fecha_creacion = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))

        return {"existe": True, "tamano_gb": tamano, "fecha_creacion": fecha_creacion}
    else:
        return {"existe": False}



## Envío de notificaciones por correo electrónico:

In [28]:
"""import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def enviar_email(asunto, cuerpo, destinatario):
    remitente = "estudio.csaluzzo@gmail.com"
    contraseña = "**********"
    
    # Configuración del mensaje
    mensaje = MIMEMultipart()
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    mensaje['Subject'] = asunto
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    # Conectar y enviar
    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(remitente, contraseña)
    texto = mensaje.as_string()
    servidor.sendmail(remitente, destinatario, texto)
    servidor.quit()

    print("Correo enviado a", destinatario)"""


'import smtplib\nfrom email.mime.multipart import MIMEMultipart\nfrom email.mime.text import MIMEText\n\ndef enviar_email(asunto, cuerpo, destinatario):\n    remitente = "estudio.csaluzzo@gmail.com"\n    contraseña = "**********"\n    \n    # Configuración del mensaje\n    mensaje = MIMEMultipart()\n    mensaje[\'From\'] = remitente\n    mensaje[\'To\'] = destinatario\n    mensaje[\'Subject\'] = asunto\n    mensaje.attach(MIMEText(cuerpo, \'plain\'))\n\n    # Conectar y enviar\n    servidor = smtplib.SMTP(\'smtp.gmail.com\', 587)\n    servidor.starttls()\n    servidor.login(remitente, contraseña)\n    texto = mensaje.as_string()\n    servidor.sendmail(remitente, destinatario, texto)\n    servidor.quit()\n\n    print("Correo enviado a", destinatario)'

Seguridad: Nunca debes exponer contraseñas directamente en el código, ya que esto pone en riesgo tus credenciales. Usa variables de entorno o servicios seguros para gestionar tus credenciales.

Acceso a Gmail: Gmail puede bloquear el acceso de aplicaciones no seguras. Es probable que necesites configurar Gmail para permitir aplicaciones menos seguras, o mejor aún, crear una contraseña de aplicaciones si tienes la verificación en dos pasos activada.


In [31]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def enviar_email(asunto, cuerpo, destinatario):
    remitente = "notificador.backup2024@gmail.com"
    contraseña = os.getenv('EMAIL_PASSWORD')  # Obtener la contraseña de las variables de entorno
    
    # Configuración del mensaje
    mensaje = MIMEMultipart()
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    mensaje['Subject'] = asunto
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    # Conectar y enviar
    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(remitente, contraseña)
    texto = mensaje.as_string()
    servidor.sendmail(remitente, destinatario, texto)
    servidor.quit()

    print("Correo enviado a", destinatario)


Para mejorar la seguridad y funcionamiento:

1- **Uso de variables de entorno** para la contraseña: Puedes almacenar tu contraseña en una variable de entorno en lugar de escribirla directamente en el código. Aquí un ejemplo de cómo hacerlo:

Puedes establecer la variable de entorno `EMAIL_PASSWORD` en tu sistema de la siguiente manera:

- En Windows:
`setx EMAIL_PASSWORD "tu_contraseña"`

- En Linux/MacOS:
`export EMAIL_PASSWORD="tu_contraseña"`

2- **Habilitar aplicaciones menos seguras en Gmail**: Si no tienes activada la verificación en dos pasos, puedes permitir que aplicaciones menos seguras accedan a tu cuenta de Gmail:
- Ve a Permitir el acceso de aplicaciones menos seguras y actívalo.

3- **Uso de contraseñas de aplicación**: Si tienes activada la autenticación de dos pasos, lo ideal sería generar una contraseña de aplicación específica para tu script, en lugar de utilizar tu contraseña principal.

## Envío de notificaciones por WhatsApp: Usando la API de Twilio:

In [8]:
# pip install twilio

from twilio.rest import Client

def enviar_whatsapp(mensaje):
    account_sid = 'tu_account_sid'
    auth_token = 'tu_auth_token'
    client = Client(account_sid, auth_token)

    message = client.messages.create(
        body=mensaje,
        from_='whatsapp:+14155238886',  # Número de Twilio
        to='whatsapp:+tu_numero'
    )
    print("WhatsApp enviado:", message.sid)

## Integración: Unir la verificación del backup con el envío de notificaciones:

In [33]:
def verificar_y_notificar():
    resultado = verificar_backup('D:\\Etsol\\Clientes\\Act Yapur\\DBSIF.FBK')

    if resultado['existe']:
        mensaje = f"Backup exitoso.\nTamaño: {resultado['tamano_gb']:.2f} GB\nFecha de creación: {resultado['fecha_creacion']}"
        enviar_email("Backup Exitoso", mensaje, "estudio.csaluzzo@gmail.com")
        # enviar_whatsapp(mensaje)
    else:
        mensaje = "ERROR: No se ha generado el archivo de backup."
        enviar_email("Backup Fallido", mensaje, "estudio.csaluzzo@gmail.com")
        # enviar_whatsapp(mensaje)
    print(mensaje)

verificar_y_notificar()


AttributeError: 'NoneType' object has no attribute 'encode'

In [14]:
# Verificar si funciona ruta de archivo

# resultado = verificar_backup('D:\Etsol\Clientes\Act Yapur\DBSIF.FBK')
"""ruta_backup = "D:\Etsol\Clientes\Act Yapur\DBSIF.FBK"
if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB
        tamano = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        # Obtener fecha de creación/modificación
        timestamp = os.path.getmtime(ruta_backup)
        fecha_creacion = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))
        print(f"existe {True} tamano_gb {tamano} fecha_creacion {fecha_creacion}")"""
        



existe True tamano_gb 6.917958736419678 fecha_creacion 2023-08-31 23:06:29


<>:4: SyntaxWarning: invalid escape sequence '\E'
<>:4: SyntaxWarning: invalid escape sequence '\E'
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_18348\3073632685.py:4: SyntaxWarning: invalid escape sequence '\E'
  ruta_backup = "D:\Etsol\Clientes\Act Yapur\DBSIF.FBK"
